In [1]:
from pickle import load
from pickle import dump
import numpy as np
import pandas as pd
from dateutil.parser import parse
import matplotlib.pyplot as plt
import os
from collections import Counter
import operator
from random import sample
import lifetimes
import seaborn as sns

In [2]:
def dataLoader(orderPath):
    # THis is the method to load data from the input files    
    orders = pd.read_csv(orderPath,encoding = "ISO-8859-1")
    return orders

In [7]:

filename = "data/final_with_entity.csv"

tweetDetails = dataLoader(filename)
tweetDetails.head()


tweetDetails['ParseDate'] = tweetDetails["dateTime"].apply(lambda x: parse(str(x)))
# Parsing the weekday
tweetDetails['Weekday'] = tweetDetails['ParseDate'].apply(lambda x: x.weekday())
# Parsing the Day
tweetDetails['Day'] = tweetDetails['ParseDate'].apply(lambda x: x.strftime("%A"))
# Parsing the Month
tweetDetails['Month'] = tweetDetails['ParseDate'].apply(lambda x: x.strftime("%B"))
# Getting the year
tweetDetails['Year'] = tweetDetails['ParseDate'].apply(lambda x: x.strftime("%Y"))
# Getting year and month together as one feature
tweetDetails['yearMonth'] = tweetDetails['Year'] + "_" +tweetDetails['Month']


tweetDetails['Date'] = tweetDetails['ParseDate'].apply(lambda x: x.strftime("%d"))

tweetDetails['Date']  = tweetDetails['Date'] .astype('float64')
 
tweetDetails.head()

c:\Python39\lib\site-packages\dateutil\parser\_parser.py:1213: UnknownTimezoneWarning: tzname PDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


,polarity,tweetId,dateTime,query,username,tweet,likes,retweets,ParseDate,Weekday,Day,Month,Year,yearMonth,Date,Entity
0,0,1467835305,Mon Apr 06 22:26:10 PDT 2009,NO_QUERY,MissLaura317,"@januarycrimson Sorry, babe!! My fam annoys m...",844,236,2009-04-06 22:26:10,0,Monday,April,2009,2009_April,6.0,@januarycrimson
1,0,1467998795,Mon Apr 06 23:11:18 PDT 2009,NO_QUERY,gerrymoth,Just been playing with the new Mobbler v0.4.0 ...,1620,81,2009-04-06 23:11:18,0,Monday,April,2009,2009_April,6.0,features
2,0,1468051981,Mon Apr 06 23:27:38 PDT 2009,NO_QUERY,Celz29,@adame ruddy money!! i like it!! still waiting...,1709,274,2009-04-06 23:27:38,0,Monday,April,2009,2009_April,6.0,money
3,0,1468120199,Mon Apr 06 23:49:29 PDT 2009,NO_QUERY,Celz29,@KristenjStewart just watched the trailer for ...,2038,153,2009-04-06 23:49:29,0,Monday,April,2009,2009_April,6.0,@KristenjStewart
4,0,1468276331,Tue Apr 07 00:41:45 PDT 2009,NO_QUERY,amazingphoebe,i miss @sebby_peek,592,42,2009-04-07 00:41:45,1,Tuesday,April,2009,2009_April,7.0,@sebby_peek


In [8]:
tweetDetails['Hour'] = tweetDetails['ParseDate'].apply(lambda x: x.strftime("%H"))
tweetDetails['Weekday'] = tweetDetails['ParseDate'].apply(lambda x: x.weekday())
tweetDetails['weekday_hour'] = tweetDetails['Day'] + "_" +tweetDetails['Hour']

In [9]:
tweetDetails.head()

,polarity,tweetId,dateTime,query,username,tweet,likes,retweets,ParseDate,Weekday,Day,Month,Year,yearMonth,Date,Entity,Hour,weekday_hour
0,0,1467835305,Mon Apr 06 22:26:10 PDT 2009,NO_QUERY,MissLaura317,"@januarycrimson Sorry, babe!! My fam annoys m...",844,236,2009-04-06 22:26:10,0,Monday,April,2009,2009_April,6.0,@januarycrimson,22,Monday_22
1,0,1467998795,Mon Apr 06 23:11:18 PDT 2009,NO_QUERY,gerrymoth,Just been playing with the new Mobbler v0.4.0 ...,1620,81,2009-04-06 23:11:18,0,Monday,April,2009,2009_April,6.0,features,23,Monday_23
2,0,1468051981,Mon Apr 06 23:27:38 PDT 2009,NO_QUERY,Celz29,@adame ruddy money!! i like it!! still waiting...,1709,274,2009-04-06 23:27:38,0,Monday,April,2009,2009_April,6.0,money,23,Monday_23
3,0,1468120199,Mon Apr 06 23:49:29 PDT 2009,NO_QUERY,Celz29,@KristenjStewart just watched the trailer for ...,2038,153,2009-04-06 23:49:29,0,Monday,April,2009,2009_April,6.0,@KristenjStewart,23,Monday_23
4,0,1468276331,Tue Apr 07 00:41:45 PDT 2009,NO_QUERY,amazingphoebe,i miss @sebby_peek,592,42,2009-04-07 00:41:45,1,Tuesday,April,2009,2009_April,7.0,@sebby_peek,00,Tuesday_00


In [10]:
rewardFull = tweetDetails.groupby(['Hour','Day','tweetId'])['username'].agg('sum').reset_index()
rewardFull

,Hour,Day,tweetId,username
0,00,Friday,1880080999,willtompsett
1,00,Friday,1880095370,CforrClemmensen
2,00,Friday,1880115124,wizdomlakers
3,00,Friday,1880214332,kyoisorange
4,00,Friday,1880301197,jemjem1983
...,...,...,...,...
1993,23,Tuesday,2013713816,CforrClemmensen
1994,23,Tuesday,2013731137,lisisilveira
1995,23,Tuesday,2013852005,I_enigma
1996,23,Tuesday,2013927855,lisisilveira


In [13]:
# Finding unique value for each of the segment 

hours = list(rewardFull.Hour.unique())
print('Hour', hours)
days = list(rewardFull.Day.unique())
print('Day',days)
tweetIds = list(rewardFull.tweetId.unique())
# print('tweetId',len(tweetIds))
usernames = list(rewardFull.username.unique())
print('username',len(usernames))

entities = list(rewardFull.entity.unique())
print('entities',entities)



Hour ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']
Day ['Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday']
username 92


AttributeError: 'DataFrame' object has no attribute 'entity'

In [12]:
# Defining some dictionaries for storing the values
countDic = {} # Dictionary to store the count of products
polDic = {} # Dictionary to store the value distribution
rewDic = {} # Dictionary to store the reward distribution
recoCount = {} # Dictionary to store the recommendation counts

In [ ]:
for entity in entities:
    for u in usernames:
        for d in days:
            for h in hours:
                # Get the subset of the data
                subset1 = rewardFull[(rewardFull['username'] == u) & (rewardFull['hours'] == h) 
                # &( rewardFull['tweetId'] == tweetIds)
                                & (rewardFull['Day'] == days)]                
                # Check if the subset is valid
                if len(subset1) > 0:
                    # Iterate through each of the subset and get the products and its quantities
                    stateId = str(u) + '_' + str(d) + '_' + str(h)
                    # Define a dictionary for the state ID
                    countDic[stateId] = {}                    
                    for i in range(len(subset1.username)):
                        countDic[stateId][subset1.iloc[i]['userAppearance']] = int(subset1.iloc[i]['Quantity'])